In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import argparse
import logging
import os,sys
from typing import Type
import random 
from tqdm import tqdm

import torch
import numpy as np
from pytorch3d.structures import Meshes
from pytorch3d.io import load_obj, save_obj
from pytorch3d.ops import GraphConv

In [3]:
from gcnna.config import Config
from gcnna.models.base_nn import GraphConvClf
from gcnna.layers.utils import unpack_mesh_attr, pack_mesh_attr
from scripts.ico_objects import ico_disk

In [4]:
def load_mesh(tst_obj):
    verts, faces, aux = load_obj(tst_obj)
    mesh = Meshes(verts=[verts], faces=[faces.verts_idx]).cuda()
    return mesh

table_pth = '/scratch/jiadeng_root/jiadeng/shared_data/datasets/ShapeNetCore.v1/04379243/1028a9cbaa7a333230bbd4cddd04c77b/model.obj'
airplane_pth = '/scratch/jiadeng_root/jiadeng/shared_data/datasets/ShapeNetCore.v1/02691156/105f7f51e4140ee4b6b87e72ead132ed/model.obj'
chair_pth = '/scratch/jiadeng_root/jiadeng/shared_data/datasets/ShapeNetCore.v1/03001627/1016f4debe988507589aae130c1f06fb/model.obj'
rifle_pth = '/scratch/jiadeng_root/jiadeng/shared_data/datasets/ShapeNetCore.v1/04090263/10439f1f81fbee202be79d8b285c1e//model.obj'
table, airplane, chair, rifle = load_mesh(table_pth), load_mesh(airplane_pth), load_mesh(chair_pth), load_mesh(rifle_pth)

In [5]:
v1, f1, _ = load_obj(table_pth)
v2, f2, _ = load_obj(airplane_pth)
mesh = Meshes(verts=[v1,v2], faces=[f1.verts_idx,f2.verts_idx]).cuda()

In [6]:
cfg = Config('config/train_clf.yml')
clf = GraphConvClf(cfg).cuda()

In [7]:
clf

GraphConvClf(
  (gconvs): ModuleList(
    (0): GraphConv(3 -> 128, directed=False)
    (1): GraphConv(128 -> 256, directed=False)
    (2): GraphConv(256 -> 512, directed=False)
  )
  (gap): GlobalAveragePooling()
  (gmp): GlobalMaxPooling()
  (fc1): Linear(in_features=1024, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [8]:
a = clf(mesh)

torch.Size([18489, 3])
torch.Size([18489, 128])
torch.Size([18489, 256])
torch.Size([18489, 512])


TypeError: sum() received an invalid combination of arguments - got (tuple, int), but expected one of:
 * (Tensor input, torch.dtype dtype)
      didn't match because some of the arguments have invalid types: ([31;1mtuple[0m, [31;1mint[0m)
 * (Tensor input, tuple of names dim, bool keepdim, torch.dtype dtype, Tensor out)
 * (Tensor input, tuple of ints dim, bool keepdim, torch.dtype dtype, Tensor out)


In [ ]:
mesh = ico_disk().cuda()

In [ ]:
def get_adjacency(verts, edges):
    V = verts.shape[0] 
    e0, e1 = edges.unbind(1)

    idx01 = torch.stack([e0, e1], dim=1)  # (sum(E_n), 2)
    idx10 = torch.stack([e1, e0], dim=1)  # (sum(E_n), 2)
    idx = torch.cat([idx01, idx10], dim=0).t()  # (2, 2*sum(E_n))

    # First, we construct the adjacency matrix,
    # i.e. A[i, j] = 1 if (i,j) is an edge, or
    # A[e0, e1] = 1 &  A[e1, e0] = 1
    ones = torch.ones(idx.shape[1], dtype=torch.float32, device=verts.device)
    A = torch.sparse.FloatTensor(idx, ones, (V, V))
    return A

def update_mesh(verts, A, idx):
    

In [ ]:
# from pytorch3d.structures.utils import packed_to_list, list_to_padded, padded_to_list, list_to_packed
verts = mesh.verts_packed()
edges = mesh.edges_packed()
V = verts.shape[0] 
verts_idx = mesh.verts_packed_to_mesh_idx()
edges_idx = mesh.edges_packed_to_mesh_idx()

attn = GraphConv(3,1).cuda()
verts = torch.tanh(attn(verts, edges))

verts_unpkd, verts_size  = unpack_mesh_attr(verts_idx, verts)
edges_unpkd, edges_size = unpack_mesh_attr(edges_idx, edges)
assert len(verts_unpkd) == len(edges_unpkd)
B = len(verts_unpkd)
# print('Unpacked: ',verts_unpkd[0].shape, verts_unpkd[1].shape, edges_unpkd[0].shape, edges_unpkd[1].shape)
print('Unpacked: ',verts_unpkd[0].shape, edges_unpkd[0].shape)

A = get_adjacency(verts, edges)
# for b in B:
_, mask_idx = torch.topk(verts_unpkd[0], int(V*0.5), dim=0, sorted=False)  
mask_idx = sorted(mask_idx)

# verts_pkd = pack_mesh_attr(verts_idx, verts_unpkd)
# edges_pkd = pack_mesh_attr(edges_idx, edges_unpkd)
# print('Packed: ',verts_pkd.shape, edges_pkd.shape)

In [124]:
verts_unpkd[0][mask_idx,:]

tensor([[0.0005],
        [0.0069],
        [0.0126],
        [0.0100]], device='cuda:0', grad_fn=<IndexBackward>)

In [126]:
mask_idx

[tensor([1], device='cuda:0'),
 tensor([6], device='cuda:0'),
 tensor([7], device='cuda:0'),
 tensor([8], device='cuda:0')]

In [128]:
A = A[, mask_idx]
A = A[mask_idx, ]

SyntaxError: invalid syntax (<ipython-input-128-c09f246b2f6a>, line 1)

In [135]:
A._indices()

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8,
         2, 8, 3, 4, 5, 6, 7, 8],
        [1, 2, 3, 4, 5, 6, 7, 8, 2, 8, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 2, 3, 4, 5, 6, 7]], device='cuda:0')

In [73]:
a,b = torch.where(torch.eye(3)==1)[0].unsqueeze(-1), torch.where(torch.eye(3)==1)[1].unsqueeze(-1)

In [75]:
torch.cat([a,b], dim=1)

tensor([[0, 0],
        [1, 1],
        [2, 2]])